In [1]:
import pandas as pd
import numpy as np
from scipy import stats
import seaborn as sns
import matplotlib.pyplot as plt
from functools import reduce
import warnings
warnings.simplefilter('ignore')
warnings.filterwarnings("ignore", module="matplotlib")

In [2]:
data = pd.read_csv('../summary_data/75bp/all_tools_TRB.csv')
data

Sample                 CDR3  nReads_TCR  nReads_MIXCR  \
0        MP-11-28-12RNA_S2      CASSVNPGGYNEQFF         6.0           0.0   
1        MP-11-28-12RNA_S2        CASIRTRNEKLFF         2.0           0.0   
2        MP-11-28-12RNA_S2       CASSPGAANTEAFF         2.0           0.0   
3        MP-11-28-12RNA_S2       CASRNQGLNTEAFF         2.0           0.0   
4        MP-11-28-12RNA_S2     CASSLTGNRAYNEQFF         2.0           0.0   
...                    ...                  ...         ...           ...   
143110  SAR-11-14-12RNA_S1    CASSSTGGSVERNIQYF         0.0           0.0   
143111  SAR-11-14-12RNA_S1       CASSLEGLANGYTF         0.0           0.0   
143112  SAR-11-14-12RNA_S1      CASSPHPDSNQPQHF         0.0           0.0   
143113  SAR-11-14-12RNA_S1      CASSLGGGGANEQFF         0.0           0.0   
143114  SAR-11-14-12RNA_S1  CASSLVAGGPRSSYNEQFF         0.0           0.0   

        nReads_IMREP  nReads_TRUST4 tissue  tissue_type  total_reads_TCR  \
0                0.0            0.0   PBMC  T_cell_rich          18617.0   
1                0.0            0.0   PBMC  T_cell_rich          18617.0   
2                0.0            0.0   PBMC  T_cell_rich          18617.0   
3                0.0            0.0   PBMC  T_cell_rich          18617.0   
4                0.0            0.0   PBMC  T_cell_rich          18617.0   
...              ...            ...    ...          ...              ...   
143110           0.0            2.0   PBMC  T_cell_rich         104779.0   
143111           0.0            2.0   PBMC  T_cell_rich         104779.0   
143112           0.0            4.0   PBMC  T_cell_rich         104779.0   
143113           0.0            2.0   PBMC  T_cell_rich         104779.0   
143114           0.0            2.0   PBMC  T_cell_rich         104779.0   

        total_reads_MIXCR  total_reads_IMREP  total_reads_TRUST4  \
0                    48.0               88.0                 0.0   
1                    48.0               88.0                 0.0   
2                    48.0               88.0                 0.0   
3                    48.0               88.0                 0.0   
4                    48.0               88.0                 0.0   
...                   ...                ...                 ...   
143110              277.0             1135.0                44.0   
143111              277.0             1135.0                44.0   
143112              277.0             1135.0                44.0   
143113              277.0             1135.0                44.0   
143114              277.0             1135.0                44.0   

        frequency_TCR  frequency_MIXCR  frequency_IMREP  frequency_TRUST4  \
0            0.000322              0.0              0.0          0.000000   
1            0.000107              0.0              0.0          0.000000   
2            0.000107              0.0              0.0          0.000000   
3            0.000107              0.0              0.0          0.000000   
4            0.000107              0.0              0.0          0.000000   
...               ...              ...              ...               ...   
143110       0.000000              0.0              0.0          0.045455   
143111       0.000000              0.0              0.0          0.045455   
143112       0.000000              0.0              0.0          0.090909   
143113       0.000000              0.0              0.0          0.045455   
143114       0.000000              0.0              0.0          0.045455   

       repertoire_type                   class  
0           polyclonal  T_cell_rich_polyclonal  
1           polyclonal  T_cell_rich_polyclonal  
2           polyclonal  T_cell_rich_polyclonal  
3           polyclonal  T_cell_rich_polyclonal  
4           polyclonal  T_cell_rich_polyclonal  
...                ...                     ...  
143110      polyclonal  T_cell_rich_polyclonal  
143111      polyclonal  T_cell_rich_polyclonal  

In [3]:
def compute_frequency(samples_data, tool, discard_zero_freq_samples):

    samples = set(samples_data['Sample'])
    
    type_observed = tool
    
    data = samples_data.rename(columns={'frequency_TCR': 'frequency_truth'})
    data = data.rename(columns={'frequency_'+type_observed: 'frequency_observed'})
    data = data.rename(columns={'nReads_TCR': 'count_truth'})
    data = data.rename(columns={'nReads_'+type_observed: 'count_observed'})
    
    data = data.loc[data['frequency_truth'] > 0] #excluding clonotypes not present in ground truth

    file = open('../summary_data/75bp/capturing_per_class/capturing_'+type_observed+'.csv',"w")
        
    file.write("th,observed_portion_frequency\n")
    
    data_per_sample = {}

    for s in samples:
        
        data_per_sample[s] = data.loc[data['Sample'] == s, ['frequency_truth','frequency_observed','count_truth','count_observed']]

    for th in np.linspace(0., 1.0, 100000):

        s_portions = []

        for s in samples: 

            k_observed=0
            k_true=0

            s_observed=0
            s_true=0

            sample_data = data_per_sample[s]
            sample_data = sample_data.loc[sample_data['frequency_truth'] >= th]
            
            for index, cdr in sample_data.iterrows():

                freq = cdr['frequency_truth']
                
                if cdr['frequency_observed'] > 0: #check if observed sample has the clonotype
                        
                    k_observed += 1.0    #number of clonotypes with frequency greater or equal than th
                    s_observed += freq   #sum of frequencies of observed greater or equal than th
                        
                k_true += 1.0 #number of clonotypes with frequency greater or equal than th
                s_true += freq #sum of frequencies of TCR greater or equal than th

            if s_true > 0:
                s_portions.append(s_observed/s_true)
            elif discard_zero_freq_samples is False:
                s_portions.append(0.)

        file.write(str(th)+","+str(np.mean(s_portions))+"\n")

    file.close()
    print('done '+tool)

Discarding samples with no clonotypes from truth to compute the average portion

In [5]:
# T_cell_rich_monoclonal samples

T_cell_rich_monoclonal = data.loc[data['class'] == 'T_cell_rich_monoclonal']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(T_cell_rich_monoclonal, tool, True)

done MIXCR
done IMREP
done TRUST4


In [6]:
# T_cell_rich_polyclonal samples

T_cell_rich_polyclonal = data.loc[data['class'] == 'T_cell_rich_polyclonal']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(T_cell_rich_polyclonal, tool, True)

done MIXCR
done IMREP
done TRUST4


In [7]:
# T_cell_poor_polyclonal samples

T_cell_poor_polyclonal = data.loc[data['class'] == 'T_cell_poor_polyclonal']

tools = ['MIXCR', 'IMREP', 'TRUST4']

for tool in tools:

    compute_frequency(T_cell_poor_polyclonal, tool, True)

done MIXCR
done IMREP
done TRUST4
